In [2]:
import os
import io
import json
import xgboost
import contextlib
import numpy as np
import pandas as pd
from tqdm import tqdm
from ultralytics import YOLO
from xgboost import XGBClassifier
from pycocotools.coco import COCO
from pycocotools.cocoeval import COCOeval

In [3]:
coco_path = "data/dataset/annotations_trainval2017/instances_train2017.json"
coco = COCO(coco_path)

id_to_filename = {img['id']: img['file_name'] for img in coco.dataset['images']}

rows = []
for ann in tqdm(coco.dataset['annotations']):
    if ann['category_id'] != 1:  # Only 'person' category
        continue
    image_id = ann['image_id']
    image_name = id_to_filename[image_id]
    annotation_id = ann['id']
    bbox = ann['bbox']
    rows.append([image_name, annotation_id, ann['category_id'], *bbox])

df = pd.DataFrame(rows, columns=['image_id', 'annotation_id', 'category_id', 'bbox_x', 'bbox_y', 'bbox_width', 'bbox_height'])
df['area'] = df['bbox_width'] * df['bbox_height']

def categorize_bbox(area):
    if area < 256: 
        return 'nano'
    elif area < 1024:
        return 'small'
    elif area < 9216:
        return 'medium'
    elif area < 40000:
        return 'large'
    else:
        return 'extra'

df['bbox_category'] = df['area'].apply(categorize_bbox)
df.to_csv("test_data/coco_bbox_labeled.csv", index=False)

loading annotations into memory...
Done (t=9.07s)
creating index...
index created!


100%|██████████| 860001/860001 [00:01<00:00, 779250.39it/s] 


In [4]:
import numpy as np
import pandas as pd
from tqdm import tqdm
from pycocotools.coco import COCO
import joblib

# Load COCO keypoints dataset
keypoints_path = "data/dataset/annotations_trainval2017/person_keypoints_train2017.json"
coco_kps = COCO(keypoints_path)

# Load trained XGBoost model
model = joblib.load('models/xgb2.pkl')

# Utility to compute angle between three points
def calculate_angle(a, b, c):
    a, b, c = np.array(a), np.array(b), np.array(c)
    ba = a - b
    bc = c - b
    cosine_angle = np.dot(ba, bc) / (np.linalg.norm(ba) * np.linalg.norm(bc) + 1e-6)
    angle = np.arccos(np.clip(cosine_angle, -1.0, 1.0))
    return np.degrees(angle)

# Reformat keypoints + calculate angles to create model input
def extract_model_features(keypoints):
    kp = np.array(keypoints).reshape(-1, 3)

    # Mapping COCO indices to feature names
    index_map = {
        'Nose': 0,
        'LShoulder': 5, 'RShoulder': 6,
        'LElbow': 7, 'RElbow': 8,
        'LWrist': 9, 'RWrist': 10,
        'LHip': 11, 'RHip': 12,
        'LKnee': 13, 'RKnee': 14,
        'LFoot': 15, 'RFoot': 16,
        'LEye': 1, 'REye': 2,
        'LEar': 3, 'REar': 4
    }

    features = []
    for part in ['Nose', 'RShoulder', 'RElbow', 'RWrist',
                 'LShoulder', 'LElbow', 'LWrist',
                 'RHip', 'RKnee', 'RFoot',
                 'LHip', 'LKnee', 'LFoot',
                 'REye', 'LEye', 'REar', 'LEar']:
        x, y = kp[index_map[part]][:2]
        features.extend([x, y])

    # Angle features
    left_kneeang = calculate_angle(kp[11][:2], kp[13][:2], kp[15][:2])
    right_kneeang = calculate_angle(kp[12][:2], kp[14][:2], kp[16][:2])
    left_hipang = calculate_angle(kp[13][:2], kp[11][:2], kp[5][:2])
    right_hipang = calculate_angle(kp[14][:2], kp[12][:2], kp[6][:2])
    features.extend([left_kneeang, right_kneeang, left_hipang, right_hipang])

    return np.array(features).reshape(1, -1)

# Predict pose for each person annotation
pose_data = []
for ann in tqdm(coco_kps.dataset['annotations']):
    if ann['num_keypoints'] < 5:
        pose, conf = 'unknown', 0.0
    else:
        try:
            features = extract_model_features(ann['keypoints'])
            pred = model.predict(features)[0]
            conf = float(max(model.predict_proba(features)[0]))
            pose = 'standing' if pred == 1 else 'sitting'
        except Exception as e:
            pose, conf = 'unknown', 0.0
    pose_data.append([ann['id'], pose, conf])

# Save predictions to CSV
pose_df = pd.DataFrame(pose_data, columns=['annotation_id', 'pose', 'confidence'])
pose_df.to_csv("test_data/pose_output.csv", index=False)


loading annotations into memory...
Done (t=3.81s)
creating index...
index created!


/home/oonik/anaconda3/envs/sce/lib/python3.11/pickle.py:1718: UserWarning: [10:41:01] WARNING: /workspace/src/collective/../data/../common/error_msg.h:82: If you are loading a serialized model (like pickle in Python, RDS in R) or
configuration generated by an older version of XGBoost, please export the model by calling
`Booster.save_model` from that version first, then load it back in current version. See:

    https://xgboost.readthedocs.io/en/stable/tutorials/saving_model.html

for more details about differences between saving model and serializing.

  setstate(state)
100%|██████████| 262465/262465 [00:55<00:00, 4749.59it/s] 


In [5]:
bbox_df = pd.read_csv("test_data/coco_bbox_labeled.csv")
pose_df = pd.read_csv("test_data/pose_output.csv")
final_df = bbox_df.merge(pose_df, on='annotation_id', how='left')
final_df.to_csv("test_data/final_coco_bbox_pose.csv", index=False)

In [6]:
csv_path = "test_data/final_coco_bbox_pose.csv"
image_dir = "data/dataset/train"
coco_ann_path = "data/dataset/annotations_trainval2017/instances_train2017.json"
output_dir = "tets_data/output"
os.makedirs(output_dir, exist_ok=True)

df = pd.read_csv(csv_path)

size_subsets = {
    "nano": df[df["bbox_category"] == "nano"],
    "small": df[df["bbox_category"] == "small"],
    "medium": df[df["bbox_category"] == "medium"],
    "large": df[df["bbox_category"] == "large"],
    "extra": df[df["bbox_category"] == "extra"]
}

pose_subsets = {
    "sitting": df[df["pose"] == "sitting"],
    "standing": df[df["pose"] == "standing"]
}

models_to_test = {
    "yolo11n" : "models/yolo11n.pt",
    "yolo11s": "models/yolo11s.pt",
    "yolo11m": "models/yolo11m.pt",
    "yolo11l": "models/yolo11l.pt",
    "yolo11x": "models/yolo11x.pt"
}

buckets = ["nano", "small", "medium", "large", "extra", "sitting", "standing"]
results = {model_name: {b: None for b in buckets} for model_name in models_to_test}

In [7]:
def generate_gt_json(subset_df, original_coco_json, output_json):
    coco = COCO(original_coco_json)
    ann_ids = subset_df["annotation_id"].tolist()
    anns = coco.loadAnns(ann_ids)
    img_ids = list({ann["image_id"] for ann in anns})
    images = coco.loadImgs(img_ids)

    gt_data = {
        "info": coco.dataset.get("info", {}),
        "licenses": coco.dataset.get("licenses", []),
        "images": images,
        "annotations": anns,
        "categories": coco.dataset["categories"]
    }

    with open(output_json, "w") as f:
        json.dump(gt_data, f)



def run_inference(subset_df, subset_name, model):
    image_ids = subset_df["image_id"].unique()
    preds = []

    for img_id in tqdm(image_ids, desc=f"Inference on {subset_name}", leave=False):
        clean_id = os.path.splitext(str(img_id))[0]
        img_path = os.path.join(image_dir, f"{clean_id}.jpg")

        if not os.path.exists(img_path):
            continue

        results = model.predict(img_path, conf=0.05, verbose=False)[0]  # suppressing verbose
        for box in results.boxes:
            cls = int(box.cls[0].item())
            conf = float(box.conf[0].item())
            if cls != 0:
                continue  # only person class
            x1, y1, x2, y2 = box.xyxy[0].tolist()
            bbox = [x1, y1, x2 - x1, y2 - y1]
            preds.append({
                "image_id": int(clean_id),
                "category_id": 1,
                "bbox": [round(x, 2) for x in bbox],
                "score": round(conf, 4)
            })

    return preds

def save_preds(preds, output_json_path):
    with open(output_json_path, "w") as f:
        json.dump(preds, f)

def evaluate_map(gt_path, dt_path, label):
    coco_gt = COCO(gt_path)
    with open(dt_path, 'r') as f:
        anns = json.load(f)
    if len(anns) == 0:
        return {"Bucket": label, "MAP": 0.0}
    coco_dt = coco_gt.loadRes(anns)
    coco_eval = COCOeval(coco_gt, coco_dt, iouType='bbox')
    coco_eval.evaluate()
    coco_eval.accumulate()
    coco_eval.summarize()
    return {"Bucket": label, "MAP": round(coco_eval.stats[0], 4)}

In [8]:
import os
import time

os.environ['YOLO_VERBOSE'] = 'False'

results = []

for model_name, model_path in models_to_test.items():
    print(f"\n🚀 Evaluating model: {model_name}")
    model = YOLO(model_path)
    row = {"Model": model_name}

    total_images = 0
    start_time = time.time()  # Start timing per model

    for bucket in buckets:
        print(f"\n🔍 Processing bucket: {bucket}")

        if bucket in size_subsets:
            subset_df = size_subsets[bucket]
        elif bucket in pose_subsets:
            subset_df = pose_subsets[bucket]
        else:
            print(f"⚠️ Bucket not found: {bucket}")
            continue

        if subset_df.empty:
            print(f"⚠️ No data for bucket: {bucket}")
            row[bucket] = 0.0
            continue

        gt_path = os.path.join(output_dir, f"{model_name}_{bucket}_gt.json")
        dt_path = os.path.join(output_dir, f"{model_name}_{bucket}_pred.json")

        generate_gt_json(subset_df, coco_ann_path, gt_path)

        preds = run_inference(subset_df, bucket, model)

        save_preds(preds, dt_path)
        metrics = evaluate_map(gt_path, dt_path, bucket)
        row[bucket] = metrics["MAP"]

        total_images += len(subset_df)

    elapsed = time.time() - start_time
    row["Total Time (s)"] = round(elapsed, 2)
    row["Images"] = total_images
    row["FPS"] = round(total_images / elapsed, 2)

    print(f"⏱️ {model_name} runtime: {elapsed:.2f}s, Images: {total_images}, FPS: {row['FPS']}")
    results.append(row)



🚀 Evaluating model: yolo11n

🔍 Processing bucket: nano
loading annotations into memory...
Done (t=9.29s)
creating index...
index created!


loading annotations into memory...
Done (t=0.16s)
creating index...
index created!
Loading and preparing results...
DONE (t=0.09s)
creating index...
index created!
Running per image evaluation...
Evaluate annotation type *bbox*
DONE (t=10.36s).
Accumulating evaluation results...
DONE (t=1.04s).
 Average Precision  (AP) @[ IoU=0.50:0.95 | area=   all | maxDets=100 ] = 0.010
 Average Precision  (AP) @[ IoU=0.50      | area=   all | maxDets=100 ] = 0.029
 Average Precision  (AP) @[ IoU=0.75      | area=   all | maxDets=100 ] = 0.004
 Average Precision  (AP) @[ IoU=0.50:0.95 | area= small | maxDets=100 ] = 0.020
 Average Precision  (AP) @[ IoU=0.50:0.95 | area=medium | maxDets=100 ] = -1.000
 Average Precision  (AP) @[ IoU=0.50:0.95 | area= large | maxDets=100 ] = -1.000
 Average Recall     (AR) @[ IoU=0.50:0.95 | area=   all | maxDets=  1 ] = 0.011
 Average Recall     (AR) @[ IoU=0.50:0.95 | area=   all | maxDets= 10 ] = 0.109
 Average Recall     (AR) @[ IoU=0.50:0.95 | area=   all | maxD

loading annotations into memory...
Done (t=0.24s)
creating index...
index created!
Loading and preparing results...
DONE (t=0.14s)
creating index...
index created!
Running per image evaluation...
Evaluate annotation type *bbox*
DONE (t=15.79s).
Accumulating evaluation results...
DONE (t=1.81s).
 Average Precision  (AP) @[ IoU=0.50:0.95 | area=   all | maxDets=100 ] = 0.062
 Average Precision  (AP) @[ IoU=0.50      | area=   all | maxDets=100 ] = 0.132
 Average Precision  (AP) @[ IoU=0.75      | area=   all | maxDets=100 ] = 0.051
 Average Precision  (AP) @[ IoU=0.50:0.95 | area= small | maxDets=100 ] = 0.234
 Average Precision  (AP) @[ IoU=0.50:0.95 | area=medium | maxDets=100 ] = -1.000
 Average Precision  (AP) @[ IoU=0.50:0.95 | area= large | maxDets=100 ] = -1.000
 Average Recall     (AR) @[ IoU=0.50:0.95 | area=   all | maxDets=  1 ] = 0.039
 Average Recall     (AR) @[ IoU=0.50:0.95 | area=   all | maxDets= 10 ] = 0.310
 Average Recall     (AR) @[ IoU=0.50:0.95 | area=   all | maxD

loading annotations into memory...
Done (t=0.53s)
creating index...
index created!
Loading and preparing results...
DONE (t=1.46s)
creating index...
index created!
Running per image evaluation...
Evaluate annotation type *bbox*
DONE (t=26.19s).
Accumulating evaluation results...
DONE (t=3.09s).
 Average Precision  (AP) @[ IoU=0.50:0.95 | area=   all | maxDets=100 ] = 0.244
 Average Precision  (AP) @[ IoU=0.50      | area=   all | maxDets=100 ] = 0.392
 Average Precision  (AP) @[ IoU=0.75      | area=   all | maxDets=100 ] = 0.266
 Average Precision  (AP) @[ IoU=0.50:0.95 | area= small | maxDets=100 ] = 0.259
 Average Precision  (AP) @[ IoU=0.50:0.95 | area=medium | maxDets=100 ] = 0.527
 Average Precision  (AP) @[ IoU=0.50:0.95 | area= large | maxDets=100 ] = -1.000
 Average Recall     (AR) @[ IoU=0.50:0.95 | area=   all | maxDets=  1 ] = 0.098
 Average Recall     (AR) @[ IoU=0.50:0.95 | area=   all | maxDets= 10 ] = 0.538
 Average Recall     (AR) @[ IoU=0.50:0.95 | area=   all | maxDe

loading annotations into memory...
Done (t=1.63s)
creating index...
index created!
Loading and preparing results...
DONE (t=0.15s)
creating index...
index created!
Running per image evaluation...
Evaluate annotation type *bbox*
DONE (t=21.62s).
Accumulating evaluation results...
DONE (t=2.92s).
 Average Precision  (AP) @[ IoU=0.50:0.95 | area=   all | maxDets=100 ] = 0.432
 Average Precision  (AP) @[ IoU=0.50      | area=   all | maxDets=100 ] = 0.554
 Average Precision  (AP) @[ IoU=0.75      | area=   all | maxDets=100 ] = 0.488
 Average Precision  (AP) @[ IoU=0.50:0.95 | area= small | maxDets=100 ] = 0.018
 Average Precision  (AP) @[ IoU=0.50:0.95 | area=medium | maxDets=100 ] = 0.482
 Average Precision  (AP) @[ IoU=0.50:0.95 | area= large | maxDets=100 ] = 0.446
 Average Recall     (AR) @[ IoU=0.50:0.95 | area=   all | maxDets=  1 ] = 0.300
 Average Recall     (AR) @[ IoU=0.50:0.95 | area=   all | maxDets= 10 ] = 0.769
 Average Recall     (AR) @[ IoU=0.50:0.95 | area=   all | maxDet

...

loading annotations into memory...
Done (t=0.54s)
creating index...
index created!
Loading and preparing results...
DONE (t=1.58s)
creating index...
index created!
Running per image evaluation...
Evaluate annotation type *bbox*
DONE (t=27.02s).
Accumulating evaluation results...
DONE (t=2.98s).
 Average Precision  (AP) @[ IoU=0.50:0.95 | area=   all | maxDets=100 ] = 0.338
 Average Precision  (AP) @[ IoU=0.50      | area=   all | maxDets=100 ] = 0.460
 Average Precision  (AP) @[ IoU=0.75      | area=   all | maxDets=100 ] = 0.389
 Average Precision  (AP) @[ IoU=0.50:0.95 | area= small | maxDets=100 ] = 0.383
 Average Precision  (AP) @[ IoU=0.50:0.95 | area=medium | maxDets=100 ] = 0.687
 Average Precision  (AP) @[ IoU=0.50:0.95 | area= large | maxDets=100 ] = -1.000
 Average Recall     (AR) @[ IoU=0.50:0.95 | area=   all | maxDets=  1 ] = 0.111
 Average Recall     (AR) @[ IoU=0.50:0.95 | area=   all | maxDets= 10 ] = 0.679
 Average Recall     (AR) @[ IoU=0.50:0.95 | area=   all | maxDe

loading annotations into memory...
Done (t=0.42s)
creating index...
index created!
Loading and preparing results...
DONE (t=0.10s)
creating index...
index created!
Running per image evaluation...
Evaluate annotation type *bbox*
DONE (t=21.63s).
Accumulating evaluation results...
DONE (t=2.76s).
 Average Precision  (AP) @[ IoU=0.50:0.95 | area=   all | maxDets=100 ] = 0.505
 Average Precision  (AP) @[ IoU=0.50      | area=   all | maxDets=100 ] = 0.591
 Average Precision  (AP) @[ IoU=0.75      | area=   all | maxDets=100 ] = 0.562
 Average Precision  (AP) @[ IoU=0.50:0.95 | area= small | maxDets=100 ] = 0.026
 Average Precision  (AP) @[ IoU=0.50:0.95 | area=medium | maxDets=100 ] = 0.610
 Average Precision  (AP) @[ IoU=0.50:0.95 | area= large | maxDets=100 ] = 0.501
 Average Recall     (AR) @[ IoU=0.50:0.95 | area=   all | maxDets=  1 ] = 0.320
 Average Recall     (AR) @[ IoU=0.50:0.95 | area=   all | maxDets= 10 ] = 0.870
 Average Recall     (AR) @[ IoU=0.50:0.95 | area=   all | maxDet

loading annotations into memory...
Done (t=0.57s)
creating index...
index created!
Loading and preparing results...
DONE (t=0.11s)
creating index...
index created!
Running per image evaluation...
Evaluate annotation type *bbox*
DONE (t=22.82s).
Accumulating evaluation results...
DONE (t=5.19s).
 Average Precision  (AP) @[ IoU=0.50:0.95 | area=   all | maxDets=100 ] = 0.829
 Average Precision  (AP) @[ IoU=0.50      | area=   all | maxDets=100 ] = 0.890
 Average Precision  (AP) @[ IoU=0.75      | area=   all | maxDets=100 ] = 0.874
 Average Precision  (AP) @[ IoU=0.50:0.95 | area= small | maxDets=100 ] = 0.000
 Average Precision  (AP) @[ IoU=0.50:0.95 | area=medium | maxDets=100 ] = 0.100
 Average Precision  (AP) @[ IoU=0.50:0.95 | area= large | maxDets=100 ] = 0.845
 Average Recall     (AR) @[ IoU=0.50:0.95 | area=   all | maxDets=  1 ] = 0.686
 Average Recall     (AR) @[ IoU=0.50:0.95 | area=   all | maxDets= 10 ] = 0.941
 Average Recall     (AR) @[ IoU=0.50:0.95 | area=   all | maxDet

loading annotations into memory...
Done (t=0.50s)
creating index...
index created!
Loading and preparing results...
DONE (t=0.12s)
creating index...
index created!
Running per image evaluation...
Evaluate annotation type *bbox*
DONE (t=24.34s).
Accumulating evaluation results...
DONE (t=3.03s).
 Average Precision  (AP) @[ IoU=0.50:0.95 | area=   all | maxDets=100 ] = 0.589
 Average Precision  (AP) @[ IoU=0.50      | area=   all | maxDets=100 ] = 0.671
 Average Precision  (AP) @[ IoU=0.75      | area=   all | maxDets=100 ] = 0.638
 Average Precision  (AP) @[ IoU=0.50:0.95 | area= small | maxDets=100 ] = -1.000
 Average Precision  (AP) @[ IoU=0.50:0.95 | area=medium | maxDets=100 ] = 0.495
 Average Precision  (AP) @[ IoU=0.50:0.95 | area= large | maxDets=100 ] = 0.708
 Average Recall     (AR) @[ IoU=0.50:0.95 | area=   all | maxDets=  1 ] = 0.376
 Average Recall     (AR) @[ IoU=0.50:0.95 | area=   all | maxDets= 10 ] = 0.840
 Average Recall     (AR) @[ IoU=0.50:0.95 | area=   all | maxDe

loading annotations into memory...
Done (t=0.60s)
creating index...
index created!
Loading and preparing results...
DONE (t=0.14s)
creating index...
index created!
Running per image evaluation...
Evaluate annotation type *bbox*
DONE (t=24.80s).
Accumulating evaluation results...
DONE (t=2.82s).
 Average Precision  (AP) @[ IoU=0.50:0.95 | area=   all | maxDets=100 ] = 0.543
 Average Precision  (AP) @[ IoU=0.50      | area=   all | maxDets=100 ] = 0.636
 Average Precision  (AP) @[ IoU=0.75      | area=   all | maxDets=100 ] = 0.604
 Average Precision  (AP) @[ IoU=0.50:0.95 | area= small | maxDets=100 ] = -1.000
 Average Precision  (AP) @[ IoU=0.50:0.95 | area=medium | maxDets=100 ] = 0.632
 Average Precision  (AP) @[ IoU=0.50:0.95 | area= large | maxDets=100 ] = 0.562
 Average Recall     (AR) @[ IoU=0.50:0.95 | area=   all | maxDets=  1 ] = 0.336
 Average Recall     (AR) @[ IoU=0.50:0.95 | area=   all | maxDets= 10 ] = 0.850
 Average Recall     (AR) @[ IoU=0.50:0.95 | area=   all | maxDe

In [10]:
results_df = pd.DataFrame(results)
results_df["Avg Time per Bucket (s)"] = (results_df["Total Time (s)"] / len(buckets)).round(2)

# Reorder and select final columns
results_df = results_df[["Model"] + buckets + ["Avg Time per Bucket (s)", "Total Time (s)", "FPS"]]

print("\n📊 Final Model Comparison Table:")
print(results_df)



📊 Final Model Comparison Table:
     Model    nano   small  medium   large   extra  sitting  standing  \
0  yolo11n  0.0098  0.0624  0.2438  0.4321  0.6932   0.4710    0.4982   
1  yolo11s  0.0221  0.0903  0.2848  0.4616  0.7489   0.5243    0.5186   
2  yolo11m  0.0332  0.1101  0.3164  0.4762  0.7892   0.5574    0.5318   
3  yolo11l  0.0352  0.1130  0.3230  0.4866  0.8107   0.5729    0.5394   
4  yolo11x  0.0414  0.1226  0.3376  0.5046  0.8291   0.5885    0.5433   

   Avg Time per Bucket (s)  Total Time (s)     FPS  
0                   251.13         1757.89  225.66  
1                   276.34         1934.39  205.07  
2                   392.40         2746.78  144.42  
3                   454.23         3179.61  124.76  
4                   708.54         4959.78   79.98  


In [11]:
results_df

,Model,nano,small,medium,large,extra,sitting,standing,Avg Time per Bucket (s),Total Time (s),FPS
0,yolo11n,0.0098,0.0624,0.2438,0.4321,0.6932,0.4710,0.4982,251.13,1757.89,225.66
1,yolo11s,0.0221,0.0903,0.2848,0.4616,0.7489,0.5243,0.5186,276.34,1934.39,205.07
2,yolo11m,0.0332,0.1101,0.3164,0.4762,0.7892,0.5574,0.5318,392.40,2746.78,144.42
3,yolo11l,0.0352,0.1130,0.3230,0.4866,0.8107,0.5729,0.5394,454.23,3179.61,124.76
4,yolo11x,0.0414,0.1226,0.3376,0.5046,0.8291,0.5885,0.5433,708.54,4959.78,79.98
